In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)
options(repr.matrix.max.rows = 6)

## **Introduction**

## **Methods & Results**

In [ ]:
# loads data using read_csv function
players_origin <- read_csv("https://raw.githubusercontent.com/mcheng250/DSCI_Project_final_report/refs/heads/main/data/players.csv")
sessions_origin <- read_csv("https://raw.githubusercontent.com/mcheng250/DSCI_Project_final_report/refs/heads/main/data/sessions.csv")

In [ ]:
# Clean up the sessions.csv: mutate start_time and end_time to get session_length, use dmy_hm to turn start_time and 
# end_time into proper format, use difftime to calculate session length
# wrangle the session_length so we have avg_session_length. 
# Clean other columns so all we left is hashedEmail and avg_session_length while rounding the avg_session_length 
# decimal place so the data looks more clean
sessions_tidy <- sessions_origin |>
                    mutate(start_time = dmy_hm(start_time),
                           end_time = dmy_hm(end_time),
                           session_length = as.numeric(difftime(end_time, start_time, units = "mins"))) |>
                    group_by(hashedEmail) |>
                    summarize(avg_session_length = round(mean(session_length, na.rm = TRUE),2))
head(sessions_tidy)

In [ ]:
# Take out Age ,subscribe and hashedEmail from the player dataset to clean up players.csv
players_tidy <- players_origin |>
                    select(Age,subscribe,hashedEmail)  
head(players_tidy)

In [ ]:
# Merge two datasets to make the final clean data
tidy_data <- players_tidy |>
                left_join(sessions_tidy, by = "hashedEmail") |>
                drop_na(avg_session_length)
head(tidy_data)

In [ ]:
# summary of dataset
# The summary of mean value, min value and max value of avg_session_length
tidy_data |> summarize(mean_avg_session_length = round(mean(avg_session_length, na.rm = TRUE),2),
                       min_avg_session_length = round(min(avg_session_length, na.rm = TRUE),2),
                       max_avg_session_length = round(max(avg_session_length, na.rm = TRUE),2))
# The summary of mean value, min value and max value of Age
tidy_data |> summarize(mean_age = round(mean(Age, na.rm = TRUE),2),
                            min_age = round(min(Age, na.rm = TRUE),2),
                            max_age = round(max(Age, na.rm = TRUE),2))
# The summary of players whose subscribe is FALSE
tidy_data |> filter(subscribe == FALSE) |> 
                summarize(subscribe_FALSE = n())
# The summary of players whose subscribe is TRUE
tidy_data |> filter(subscribe == TRUE) |> 
                summarize(subscribe_TRUE = n())
# table of mean value summary
tidy_data |> summarize(mean_avg_session_length = round(mean(avg_session_length, na.rm = TRUE),2),
                                        mean_age = round(mean(Age, na.rm = TRUE),2)) |>
                              pivot_longer(cols = everything(),
                                           names_to = "variable", 
                                           values_to = "mean_value")

In [ ]:
#convert Subscribe data to factor
tidy_data <- tidy_data |>
  mutate(subscribe = factor(subscribe, levels = c(TRUE, FALSE), 
                          labels = c("Yes", "No"))) |>   
drop_na(Age, avg_session_length, subscribe)


tidy_data

In [ ]:
set.seed(3456) 
# Randomly take 75% of the data for the training set, and 25% for testing set
data_split <- initial_split(tidy_data, prop = 0.75, strata = subscribe)  
train_data <- training(data_split)   
test_data <- testing(data_split)
# Scaling the variable and create recipe
knn_recipe <- recipe(subscribe ~ Age + avg_session_length, data = train_data) |>
            step_scale(all_predictors()) |>
            step_center(all_predictors())

knn_spec <- nearest_neighbor(weight_func = "rectangular", 
  neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

In [ ]:
knn_workflow <- workflow() |>
  add_recipe(knn_recipe) |>
  add_model(knn_spec)

In [ ]:
# 10-fold cross-validation
train_vfolds <- vfold_cv(train_data, v = 10, strata = subscribe)

In [ ]:
k_values <- tibble(neighbors = seq(1, 30))

# Tune to find best k
knn_results <- knn_workflow |>
  tune_grid(resamples = train_vfolds, grid = k_values) |>
  collect_metrics()

# Find k with highest accuracy
best_k <- knn_results |>
  filter(.metric == "accuracy") |>
  arrange(desc(mean)) |>
  slice(1)

best_k

In [ ]:
# Visualization of K tuning results
knn_results |>
  filter(.metric == "accuracy") |>
  ggplot(aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  geom_vline(xintercept = best_k_value, linetype = "dashed", color = "red") +
  labs(x = "Number of Neighbors (K)",
       y = "Cross-Validation Accuracy",
       title = "Model Accuracy vs. K Value") +
  theme_minimal()

Based upon our calculations our most optimal K would be 7. 

In [ ]:
best_k_value <- best_k |> pull(neighbors)

final_knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k_value) |>
  set_engine("kknn") |>
  set_mode("classification")

final_workflow <- workflow() |>
  add_recipe(knn_recipe) |>
  add_model(final_knn_spec)

final_fit <- final_workflow |>
  fit(data = train_data)

In [ ]:
test_predictions <- final_fit |>
  predict(test_data) |>
  bind_cols(test_data)

head(test_predictions)

In [ ]:
test_predictions |>
  conf_mat(truth = subscribe, estimate = .pred_class)

test_accuracy <- test_predictions |>
  metrics(truth = subscribe, estimate = .pred_class) |>
  filter(.metric == "accuracy")

test_precision <- test_predictions |>
  precision(truth = subscribe, estimate = .pred_class)

test_precision

In [ ]:
cat("Best K:", best_k_value, "\n")
cat("Cross-validation accuracy:", round(best_k$mean, 3), "\n")
cat("Test set accuracy:", round(test_accuracy$.estimate, 3), "\n")

In [ ]:
  ggplot(aes(x = Age, y = avg_session_length)) +
  geom_point(aes(color = .pred_class, shape = subscribe), size = 3, alpha = 0.7) +
  labs(x = "Age",
       y = "Average Session Length (minutes)",
       color = "Predicted Subscription",
       shape = "Actual Subscription",
       title = "KNN Model Predictions vs Actual Subscription Status",
       subtitle = paste("K =", best_k_value)) +
  scale_color_manual(values = c("Yes" = "blue", "No" = "red")) +
  theme_minimal() +
  theme(legend.position = "right")

## **Discussion**

## **References**